# Xenplate Communication Code

In [1]:
import requests
from datetime import datetime
import time
import json

In [2]:
#codes for convenience
user = "GroupThree"
key = "wRjUm82tln5ZYPpd1tQ3rV0c"
certif = "N3s7i2uYJ0pgAiRxEHv2FNYw"

path = r"C:\Users\Alex\Documents\Cambridge\IIA\3rd Year Projects\3GM1 Physio Project\Xenplate Passwords\L2S2-2018-CUEDGroup3-20180509"

cert_path = path + ".crt"
key_path = path + ".key.decrypted"

In [3]:
print(datetime.fromtimestamp(1346236702))
s = "01/12/2011"
t_stamp = time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()) + 5364662400 #add seconds from 00:00:00 01/01/1800 to epoch (1970)
t_now = datetime.now().timestamp() + 5364662400
print(t_stamp)
print(t_now)

2012-08-29 11:38:22
6687360000.0
6892423921.136826


# Record (Key plate/Person)

'id_number' is the user-assigned unique identifier, for instance the patient's NHS number - you will need to provide that one
the record property names are only in the form 'IdNumber' when calling the record search function, 
because of an inconsistency in the code - everywhere else, use the format specified in the documentation ('id_number')

'id' is the unique identifier that we assign to an entry in the database:
when calling any 'create' function, you don't need to send a value for that and if you do, we'll ignore it. 
However when you read items, we'll send you the id, since that's the quickest way to uniquely identify the thing you're reading.

#### Create Record/Key plate

In [4]:
#record_create = requests.post('https://cued2018.xenplate.com/api/record/create',
#                               json={"record": {
#                                       "first_names": "Alex",
#                                       "surname": "Darch",
#                                       "address_line1": "18 Thornton Ave",
#                                       "address_line2": "Warsash",
#                                       "address_town": "Southampton",
#                                       "address_county": "Hampshire",
#                                       "address_postcode": "SO31 9FJ",
#                                       "id_number": "1234567890",               #assign this
#                                       "date_of_birth": t_now,
#                                       "contact_email": "ad806@cam.ac.uk",
#                                       "contact_mobile": "",
#                                       "contact_phone": ""
#                                       }
#                                     },
#                                 headers={'Authorization': 'X-API-KEY: ' + key},  #dont need the certificate password as we have unlocked it
#                                 cert=(cert_path, key_path)                       #not verify, cert for all of them
#                               )
#attrs = vars(record_create)
#print('\n'.join("%s: %s" % item for item in attrs.items()))                    #If we use print(stuff.text) it is the equivelent of print(stuff._content)                  

#### Search Records

In [16]:
record_search = requests.post('https://cued2018.xenplate.com/api/record/search',
                        json={"filters": [{
                                        "operator": 4,          #comparison operator. 1 = "=", 2 = "<", 4 = ">". (There is no 3, and 0 is invalid)
                                        "property": "IdNumber", #Inconsistency in code means "records" uses IdNumber, but the others use id_number
                                        "value": 7000000000     #can have a number or a string here
                                        }]
                            },
                        headers={'Authorization': 'X-API-KEY: ' + key},
                        cert=(cert_path, key_path)                          #dont need the certificate password as we have unlocked it
                     )
attrs = vars(record_search)
print('\n'.join("%s: %s" % item for item in attrs.items()))


_content: b'{"RecordSearchResult":{"status":0,"records":[{"date_of_birth":6483196800,"first_names":"Martha","id":38,"id_number":"9990082235","surname":"Patterson"},{"date_of_birth":5762188800,"first_names":"Jeremy","id":40,"id_number":"9990082200","surname":"Thomas"}]}}'
_content_consumed: True
_next: None
status_code: 200
headers: {'Content-Length': '258', 'Content-Type': 'application/json; charset=utf-8', 'Server': 'Microsoft-HTTPAPI/2.0', 'Date': 'Thu, 31 May 2018 08:48:29 GMT'}
raw: <urllib3.response.HTTPResponse object at 0x0000025941297E80>
url: https://cued2018.xenplate.com/api/record/search
encoding: utf-8
history: []
reason: OK
cookies: <RequestsCookieJar[]>
elapsed: 0:00:00.247160
request: <PreparedRequest [POST]>
connection: <requests.adapters.HTTPAdapter object at 0x0000025941297978>


#### Read Specific Record

In [6]:
#need to search for record_id, not IdNumber/id_number
record_read_by_id = requests.get('https://cued2018.xenplate.com/api/record/read?record_id=13', #only works for record_id (i.e. not id_number, first_names... etc.)
                        headers={'Authorization': 'X-API-KEY: ' + key},
                        cert=(cert_path, key_path)                          #dont need the certificate password as we have unlocked it
                     )                                                      #is there a particular reason why cert uses curved brackets rather than curly ones?
attrs = vars(record_read_by_id)
print('\n'.join("%s: %s" % item for item in attrs.items()))

_content: b'{"RecordReadResult":{"status":0,"record":{"address_county":"Hampshire","address_line1":"18 Thornton Ave","address_line2":"Warsash","address_postcode":"SO31 9FJ","address_town":"Southampton","contact_email":"ad806@cam.ac.uk","contact_mobile":"","contact_phone":"","date_of_birth":1322697600,"first_names":"Alex","id":13,"id_number":"1234567890","surname":"Darch"}}}'
_content_consumed: True
_next: None
status_code: 200
headers: {'Content-Length': '364', 'Content-Type': 'application/json; charset=utf-8', 'Server': 'Microsoft-HTTPAPI/2.0', 'Date': 'Wed, 30 May 2018 20:38:38 GMT'}
raw: <urllib3.response.HTTPResponse object at 0x00000225E3485B00>
url: https://cued2018.xenplate.com/api/record/read?record_id=13
encoding: utf-8
history: []
reason: OK
cookies: <RequestsCookieJar[]>
elapsed: 0:00:00.208098
request: <PreparedRequest [GET]>
connection: <requests.adapters.HTTPAdapter object at 0x00000225E34722B0>


# Plate Data

#### Create Plate

'control_values' are the actual values of data in your filled-in form - get a plate template and have a look at what's inside it and hopefully this will make sense. The control_values collection is a dictionary of control id to value, e.g. if your plate template has a 'dosage quantity' field in it, and its id is 427, and you want to post a value of 100 for that field, then the control_values entry for that would be { "id": "427", "value": "100"}. If you can download some plate data you'll see how it's laid out and that will guide you.

'plate_template_id' is 'id' for a plate template - you can get this by searching for templates (you don't have this call in your code but it's in the doc - /template/list/all will get you all of them); likewise the version (templates are versioned and you should make sure you're using the latest)

plate_id is 'id' for a plate data instance - you can get this by searching for plates (/api/data/list)

In [7]:
plate_data_create = requests.post('https://cued2018.xenplate.com/api/data/create',
                        json={"data": {
                                        "record_id": 5,
                                        "plate_template_id": "2ab6af16-1d0b-4daf-8c23-6e61fccdbfea",          
                                        "plate_template_version": 4,          #keep this as the latest iteration of leg raise template
#                                        "prior_data_id": "",                  #optional
#                                        "track_id": "",
                                        "control_values":                       
                                        [
                                              {"id":187,"value":"2018-05-17"},
                                              {"id":188,"value":"10:00"},
                                              {"id":191,"value":"1"},  
                                              {"id":192,"value":"2"},
                                              {"id":193,"value":"5"},
                                              {"id":194,"value":"1"},
                                              {"id":196,"value":"Test 2.\u000a\u000aCan I read different plates?"}
                                            ]
                                       }
                            },
                        headers={'Authorization': 'X-API-KEY: ' + key},
                        cert=(cert_path, key_path)
                     )
print(plate_data_create.text)

{"PlateDataCreateResult":{"status":0,"control_validation_errors":[],"plate_data_id":1327,"validation_errors":[]}}


In [8]:
#this is bad code, if more time, use regular expressions
def get_plate_id(text):
    #pick out and store the plate_data_id from when we create the plate
    i = 0
    while text[i] != "":
        if i < len(text) - 17: #stops it going to the end to prevent errors
            if text[i:i+16] == '"plate_data_id":':
                idnum = text[i+16:i+20]
        
        else: break

        i = i+1
    
    return idnum

plate_id = get_plate_id(plate_data_create.text)
print(plate_id)

1327


#### Read Latest instances of each plate type of a certain record

In [9]:
plate_data_read_newest = requests.get('https://cued2018.xenplate.com/api/data/list/newest?record_id=5',
                               headers={'Authorization': 'X-API-KEY: ' + key},
                               cert=(cert_path, key_path)
                               )
print(plate_data_read_newest.text)

{"PlateDataReadNewestForRecordResult":{"status":0,"plate_data_headers":[{"created_date":6890543262,"id":1009,"plate_id":"e1dfbae8-f39f-40e4-9073-2878bc5b1e96","plate_name":"Key Plate","track_id":"e1dfbae8-f39f-40e4-9073-2878bc5b1e96"},{"created_date":6892375121,"id":1327,"plate_id":"2ab6af16-1d0b-4daf-8c23-6e61fccdbfea","plate_name":"Blood Sugar Reading","track_id":"2ab6af16-1d0b-4daf-8c23-6e61fccdbfea"},{"created_date":6892372496,"id":1325,"plate_id":"3a5c9958-dc82-4c74-be84-ded2514fd9d8","plate_name":"Knee lift session log","track_id":"3a5c9958-dc82-4c74-be84-ded2514fd9d8"}]}}


In [10]:
def get_newest_plate_id(text):
    i = 0
    while text[i] != "":
        if i < len(text) - 17:
            
            if text[i:i+5] == '"id":':
                plate_id = text[i+5:i+9]
        else: break
        i = i+1
    
    return plate_id
    
print(get_newest_plate_id(plate_data_read_newest.text))

1325


#### Read specific plate data (by record ID and plate ID)

In [11]:
plate_data_read = requests.get('https://cued2018.xenplate.com/api/data/read/id?record_id=5&plate_data_id=1056', #1078 for knee
                                headers={'Authorization': 'X-API-KEY: ' + key},         #1015 for blood sugar
                                cert=(cert_path, key_path)
                                )
print(plate_data_read.text)

{"PlateDataReadByIdResult":{"status":0,"plate_data":{"control_values":[{"attachments":[],"id":4,"value":"5"},{"attachments":[],"id":9,"value":"True"},{"attachments":[],"id":10,"value":"54"},{"attachments":[],"id":13,"value":"False"},{"attachments":[],"id":14,"value":"True"},{"attachments":[],"id":15,"value":"False"},{"attachments":[],"id":16,"value":""},{"attachments":[],"id":17,"value":"32.8"},{"attachments":[],"id":18,"value":""},{"attachments":[],"id":22,"value":"12"},{"attachments":[],"id":23,"value":""},{"attachments":[],"id":24,"value":"5"},{"attachments":[],"id":25,"value":""},{"attachments":[],"id":28,"value":"8.2%"},{"attachments":[{"description":"","key":"24DAADF7B601DD71AA7B74EB2D6BA796F30F305D.png","original_file_name":"SampleKneeLiftChart.png","saved_date_time":6891911465}],"id":29,"value":""},{"attachments":[{"description":"","key":"3921BD1A1D625E3E6CEBBCD57DC99C3094F7ACF9.docx","original_file_name":"TestData.docx","saved_date_time":6891911498}],"id":32,"value":""}],"id":

In [12]:
def get_plate_template_id_and_version(text):
    #pick out and store the plate_data_id from when we create the plate

    i = 0
    while text[i] != "":
        if i < len(text) - 17: #stops it going to the end to prevent errors
            if text[i:i+20] == '"plate_template_id":':
                temp_id = text[i+21:i+57]
                
            if text[i:i+25] == '"plate_template_version":':
                version_num = text[i+25:i+27]
        else: break
        i = i+1
    
    return temp_id, version_num

print(get_plate_template_id_and_version(plate_data_read.text))

('3a5c9958-dc82-4c74-be84-ded2514fd9d8', '15')


# Plate Template

Regarding what you can store: text fields (single- or multi-line), number fields, image/file attachments, true/false fields (i.e. tick boxes) - there are others but these should cover all your bases I would think. If you need anything more complicated then get in touch. Just a simple Word document to give me an idea of content and layout (and any static images, colours, etc for decoration if you want to add those).

I've now created this plate - it's called "Knee lift session log", current version number is 11. If you have a look on the Group Three test record you will see I've created an instance with some sample data. Please feel free to have a play - note that at present the UI restricts the different types of file that can be uploaded (and in particular forbids CSV and TXT files - I've asked for this restriction to be lifted and hopefully it will be today), but the API does not - you should be able to upload anything you like via the API.

In [13]:
plate_template_read = requests.get('https://cued2018.xenplate.com/api/template/list/active', #what is the plateid? and could it be IdPlate?
                               headers={'Authorization': 'X-API-KEY: ' + key},          #also what is a track_id?
                               cert=(cert_path, key_path)
                               )
attrs = vars(plate_template_read)
print('\n'.join("%s: %s" % item for item in attrs.items())) 

_content: b'{"PlateTemplateReadAllActiveResult":{"status":0,"plate_template_headers":[{"id":"2ab6af16-1d0b-4daf-8c23-6e61fccdbfea","is_active":true,"name":"Blood Sugar Reading","version":4},{"id":"e1dfbae8-f39f-40e4-9073-2878bc5b1e96","is_active":true,"name":"Key Plate","version":47},{"id":"3a5c9958-dc82-4c74-be84-ded2514fd9d8","is_active":true,"name":"Knee lift session log","version":17}]}}'
_content_consumed: True
_next: None
status_code: 200
headers: {'Content-Length': '382', 'Content-Type': 'application/json; charset=utf-8', 'Server': 'Microsoft-HTTPAPI/2.0', 'Date': 'Wed, 30 May 2018 20:38:57 GMT'}
raw: <urllib3.response.HTTPResponse object at 0x00000225E34BCE10>
url: https://cued2018.xenplate.com/api/template/list/active
encoding: utf-8
history: []
reason: OK
cookies: <RequestsCookieJar[]>
elapsed: 0:00:00.188562
request: <PreparedRequest [GET]>
connection: <requests.adapters.HTTPAdapter object at 0x00000225E34BC5F8>


In [14]:
plate_template_read = requests.get('https://cued2018.xenplate.com/api/template/read/full?plate_name=Knee lift session log&version=17', #what is the plateid? and could it be IdPlate?
                               headers={'Authorization': 'X-API-KEY: ' + key},          #also what is a track_id?
                               cert=(cert_path, key_path)
                               )
print(plate_template_read.text)
#print(json.dumps(plate_template_read.text, sort_keys=False, indent=10, separators=(',', '\n')))

{"PlateTemplateReadByIdNameVersionResult":{"status":0,"plate_template":{"controls":[{"default":"","id":2,"name":"Panel1","type":30},{"default":"","id":3,"name":"Label1","type":1010},{"default":"","id":4,"is_editable":true,"is_numeric":true,"name":"NumberOfSessions","type":2050},{"default":"","id":5,"name":"Label2","type":1010},{"default":"","id":6,"name":"Label3","type":1010},{"default":"","id":7,"name":"Label4","type":1010},{"default":"","id":8,"name":"Label5","type":1010},{"default":"","id":9,"is_editable":true,"name":"CalfContractedYN","type":2000},{"default":"","id":10,"is_editable":true,"is_numeric":true,"name":"CalfContractedTime","type":2050},{"default":"","id":12,"name":"Label6","type":1010},{"default":"","id":13,"is_editable":true,"name":"ThighContractedYN","type":2000},{"default":"","id":14,"is_editable":true,"name":"LowerLegAngleYN","type":2000},{"default":"","id":15,"is_editable":true,"name":"ThighAngleYN","type":2000},{"default":"","id":16,"is_editable":true,"is_numeric":t

#  Files (Images and Text)

When uploading any files to Xenplate using the API, you'll need to use the File service to send them up before sending the plate data; the File/Create service call will upload a file and return a file ID, which is a long string of letters and numbers with the original file extension on the end (e.g. 1ad78b796d8a23b409da821b3d078a96bd234.jpg). Once you have this ID, that's what you put in the plate data in the 'attachments' array, along with the original file name. This will make more sense if you get some data out of Xenplate and have a look at it.

Note that due to a weird bug in Windows/.NET that one of the other groups experienced, you'll need to call File/Connect first, to establish an authenticated connection, then immediately after you can use File/Create to upload. If you only hit File/Upload then it's likely to freeze and then time out. Just make a GET call to /file/connect to do this; no parameters and no return values, just the usual headers and client certificate.

In [4]:
plate_data_read = requests.get('https://cued2018.xenplate.com/api/data/read/id?record_id=5&plate_data_id=1056', #1078 for knee
                                headers={'Authorization': 'X-API-KEY: ' + key},         #1015 for blood sugar
                                cert=(cert_path, key_path)
                                )
print(plate_data_read.text)

{"PlateDataReadByIdResult":{"status":0,"plate_data":{"control_values":[{"attachments":[],"id":4,"value":"5"},{"attachments":[],"id":9,"value":"True"},{"attachments":[],"id":10,"value":"54"},{"attachments":[],"id":13,"value":"False"},{"attachments":[],"id":14,"value":"True"},{"attachments":[],"id":15,"value":"False"},{"attachments":[],"id":16,"value":""},{"attachments":[],"id":17,"value":"32.8"},{"attachments":[],"id":18,"value":""},{"attachments":[],"id":22,"value":"12"},{"attachments":[],"id":23,"value":""},{"attachments":[],"id":24,"value":"5"},{"attachments":[],"id":25,"value":""},{"attachments":[],"id":28,"value":"8.2%"},{"attachments":[{"description":"","key":"24DAADF7B601DD71AA7B74EB2D6BA796F30F305D.png","original_file_name":"SampleKneeLiftChart.png","saved_date_time":6891911465}],"id":29,"value":""},{"attachments":[{"description":"","key":"3921BD1A1D625E3E6CEBBCD57DC99C3094F7ACF9.docx","original_file_name":"TestData.docx","saved_date_time":6891911498}],"id":32,"value":""}],"id":

In [7]:
#need to connect first because of a bug in the code
session = requests.Session()
file_connect = session.get('https://cued2018.xenplate.com/api/file/connect',
                               headers={'Authorization': 'X-API-KEY: ' + key},
                               cert=(cert_path, key_path)
                              )
print(file_connect.text)

{}


In [8]:
#add stupid '\\' to end of string to get backslash
path = r"C:\Users\Alex\Documents\Cambridge\IIA\3rd Year Projects\3GM1 Physio Project\File upload.jpg"

#print(path + '\\' + file)

with open(path, 'rb') as picture: #have to convert to rb to stream the picture in
    file_create = session.post('https://cued2018.xenplate.com/api/file/create', 
                                data = picture, 
                                headers={'Authorization': 'X-API-KEY: ' + key},
                                cert=(cert_path, key_path)
                               )
    print(file_create.text)

    
#print(upload_files(path, file))

{"FileCreateResult":{"status":0,"file_id":"F76E8B030BF3BBCC24D421441CA3CE1BD2E29E55"}}


In [9]:
#this is bad code, if more time, use regular expressions
def get_file_id(text):
    #pick out and store the plate_data_id from when we create the plate
    i = 0
    while text[i] != "":
        if i < len(text) - 17: #stops it going to the end to prevent errors
            if text[i:i+10] == '"file_id":':
                idnum = text[i+11:i+51]
        
        else: break

        i = i+1
    
    return idnum

file_id = get_file_id(file_create.text)
print(file_id)

F76E8B030BF3BBCC24D421441CA3CE1BD2E29E55


# Knee Session Code

In [10]:
class Plate(object):
    """Class representing a plate record to send"""
    def __init__(self, rec_id, temp_id, version, control_values, leg_raise_sessions, data_plot): #, raw_data
        self.record_id = rec_id
        self.temp_id = temp_id
        self.version = version
        self.control_values = control_values
        self.leg_raise_sessions = leg_raise_sessions
        self.data_plot = data_plot
#        self.raw_data = raw_data

    def to_json_dict(self):
        # append each input to plate_data_create to the control_json string ( .to_json_dict() ) note-this is a dictionary
        control_json = [self.leg_raise_sessions.to_json_dict()]
        for c in self.control_values:
            control_json += c.to_json_list()

        control_json += [self.data_plot.to_json_dict()]
#        control_json += [self.raw_data.to_json_dict()]

        return {
            "data":{
                "record_id": self.record_id,
                "plate_template_id": self.temp_id,
                "plate_template_version": self.version,
                "control_values": control_json
            }
        }


class LegRaiseSessions(object):
    """docstring for LegRaiseSessions"""
    def __init__(self, c):
        self.count = c

    def to_json_dict(self):
        return {
            "id": 4,
            "value": self.count
        }

class Calf(object):
    """control value class"""
    def __init__(self, c, t, a):
        self.complete = c
        self.time = t
        self.attempts = a

    def to_json_list(self):
        return [
            {"id": 9, "value": str(self.complete)},
            {"id": 10,"value": self.time},
            {"id": 22,"value": self.attempts}
        ]
    
class Thigh(object):
    """control value class"""
    def __init__(self, c, t, a):
        self.complete = c
        self.time = t
        self.attempts = a

    def to_json_list(self):
        return [
            {"id": 13, "value": str(self.complete)},
            {"id": 16,"value": self.time},
            {"id": 23,"value": self.attempts}
        ]
    
class Straight(object):
    """control value class"""
    def __init__(self, c, t, a):
        self.complete = c
        self.time = t
        self.attempts = a

    def to_json_list(self):
        return [
            {"id": 14, "value": str(self.complete)},
            {"id": 17,"value": self.time},
            {"id": 24,"value": self.attempts}
        ]

class Angle(object):
    """control value class"""
    def __init__(self, c, t, a):
        self.complete = c
        self.time = t
        self.attempts = a

    def to_json_list(self):
        return [
            {"id": 15, "value": str(self.complete)},
            {"id": 18,"value": self.time},
            {"id": 25,"value": self.attempts}
        ]

class data_plot(object):
    """control value class"""
    def __init__(self, k, n):
        self.key = k
        self.name = n

    def to_json_dict(self):
        return {"attachments":[{"description":"", "key":str(self.key), 
                                "original_file_name":str(self.name), "saved_date_time":int(t_now)}], 
            "id": 29, "value": ""}
    
#class raw_data(object):
#    """control value class"""
#    def __init__(self, k, n):
#        self.key = k
#        self.name = n
#
#    def to_json_dict(self):
#        return {"attachments":[{"description":"", "key":str(self.key), 
#                                "original_file_name":str(self.name), "saved_date_time":t_now}], 
#            "id": 32, "value": ""}

In [11]:
## Example "discription":str(self.discription), , "original_file_name":str(self.name)+".png"

lr = LegRaiseSessions(5)
c = [
    Calf(True, 20, 10),
    Thigh(True, 19, 3), 
    Straight(True, 18, 8),
    Angle(True, 17, 7)
]
p = Plate("13", "3a5c9958-dc82-4c74-be84-ded2514fd9d8", "17", c, lr, data_plot(file_id, 'File upload.jpg'))
#print(p.to_json_dict())
#, raw_data(file_id2, "Poster & Presentation notes")

def print_stuff(j):
    print(j.to_json_dict())
    
print_stuff(p)

{'data': {'record_id': '13', 'plate_template_id': '3a5c9958-dc82-4c74-be84-ded2514fd9d8', 'plate_template_version': '17', 'control_values': [{'id': 4, 'value': 5}, {'id': 9, 'value': 'True'}, {'id': 10, 'value': 20}, {'id': 22, 'value': 10}, {'id': 13, 'value': 'True'}, {'id': 16, 'value': 19}, {'id': 23, 'value': 3}, {'id': 14, 'value': 'True'}, {'id': 17, 'value': 18}, {'id': 24, 'value': 8}, {'id': 15, 'value': 'True'}, {'id': 18, 'value': 17}, {'id': 25, 'value': 7}, {'attachments': [{'description': '', 'key': 'F76E8B030BF3BBCC24D421441CA3CE1BD2E29E55', 'original_file_name': 'File upload.jpg', 'saved_date_time': 6892423921}], 'id': 29, 'value': ''}]}}


In [12]:
def create_plate(controls):
    plate_data_create = requests.post('https://cued2018.xenplate.com/api/data/create',
                            json=controls.to_json_dict(),
                            headers={'Authorization': 'X-API-KEY: ' + key},
                            cert=(cert_path, key_path)
                            )
    print(plate_data_create.text)
    
create_plate(p)


{"PlateDataCreateResult":{"status":0,"control_validation_errors":[],"plate_data_id":1355,"validation_errors":[]}}


### Redundant initial code (problem was that you couldnt easily assign control values)

In [2]:
path1 = r"C:\Users\Alex\Documents\Cambridge\IIA\3rd Year Projects\3GM1 Physio Project"
file1 = "Files.png"

out_text1 = upload_files(path1, file1)
print(out_text1)

file_id1 = get_file_id(out_text1)
print(file_id1)


path2 = r"C:\Users\Alex\Documents\Cambridge\IIA\3rd Year Projects\3GM1 Physio Project"
file2 = "Poster & Presentation notes.docx"

out_text2 = upload_files(path2, file2)
print(out_text2)

file_id2 = get_file_id(out_text2)
print(file_id2)

NameError: name 'upload_files' is not defined

In [ ]:
                                {"data": {
                                            "record_id": 5,
                                        "plate_template_id": "3a5c9958-dc82-4c74-be84-ded2514fd9d8", #what is a plate template id?
                                        "plate_template_version": 15,          #keep this as the latest iteration of leg raise template
#                                        "prior_data_id": "",                  #optional
#                                        "track_id": "",
                                        "control_values":                       
                                        [
                                                {"id": "4", "value": 10}, #leg raise sessions
                                            #first green bubble
                                                {"id": 9, "value": True},  #completed? can be True or "True", (string or bool)
                                                {"id": 10, "value": 17.3}, #best time
                                                {"id": 22, "value": 12},   #attempts
                                            #second green bubble
                                                {"id": 13, "value": "True"},  #completed?
                                                {"id": 16, "value": 17.3}, #best time
                                                {"id": 23, "value": 12},   #attempts
                                            #third green bubble
                                                {"id": 14, "value": "True"},  #completed?
                                                {"id": 17, "value": 17.3}, #best time
                                                {"id": 24, "value": 12},   #attempts
                                            #fourth green bubble
                                                {"id": 15, "value": "True"},  #completed?
                                                {"id": 18, "value": 17.3}, #best time
                                                {"id": 25, "value": 12},   #attempts
                                            
                                            #comparison to last session's tasks
                                                {"id": 28, "value": "10%"},
                                            #figure and raw data attachment?
                                        ]
                                       }
                            }